In [9]:
import numpy as np
from IPython.display import clear_output
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    Box,
    Cylinder,
    DiagramBuilder,
    InverseKinematics,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    RigidTransform,
    Role,
    RollPitchYaw,
    RotationMatrix,
    Solve,
    StartMeshcat,
    RevoluteJoint
)

from pydrake.all import (
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator)

In [20]:
def AddPanda(plant, pandaFile, qs=[0., 0., 0., 0., 0., 0., 0]):
    parser = Parser(plant)
    panda = parser.AddModels(pandaFile)[0]
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("panda_link0"))

    # Set default positions:
    index = 0
    for joint_index in plant.GetJointIndices(panda):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(qs[index])
            index += 1
    return panda

In [28]:
# Start the visualizer.
meshcat = StartMeshcat()
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
pandaFile = "/Users/xiao/0_codes/ICBM_drake/models/franka_description/urdf/panda_arm_hand.urdf"
panda = AddPanda(plant, pandaFile, [0.1, -0.1,  1.5, -1.8,  0.3,  1.3,  0.5])
plant.Finalize()

meshcat.Delete()
meshcat.DeleteAddedControls()
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat, MeshcatVisualizerParams())
collision = MeshcatVisualizer.AddToBuilder(
    builder,
    scene_graph,
    meshcat,
    MeshcatVisualizerParams(prefix="collision", role=Role.kProximity, visible_by_default=False),
)

INFO:drake:Meshcat listening for connections at http://localhost:7006


In [29]:
diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)

In [30]:
q0 = plant.GetPositions(plant_context)
# gripper_frame = plant.GetFrameByName("body", wsg) # this is to add the gripper to robot
gripper_frame = plant.GetFrameByName("panda_rightfinger")
print(q0)

[ 0.1 -0.1  1.5 -1.8  0.3  1.3  0.5  0.   0. ]


In [63]:
def InvKinProg(context, pose):
    ik = InverseKinematics(plant, plant_context)
    ik.AddPositionConstraint(
        gripper_frame,
        [0, 0, 0],
        plant.world_frame(),
        pose.translation(),
        pose.translation(),
    )
    ik.AddOrientationConstraint(
        gripper_frame,
        RotationMatrix(),
        plant.world_frame(),
        pose.rotation(),
        0.0,
    )
    prog = ik.get_mutable_prog()
    q = ik.q()
    prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
    prog.SetInitialGuess(q, q0)
    result = Solve(ik.prog())
    if result.is_success():
        print("IK success")
    else:
        print("IK failure")
    return result

In [64]:
pose = RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 0.0, 1.0],
    [0.0, -1.0, 0.0],
  ]),
  p=[-0.1, -0.2, 0.104],
)

objPose_in_w = RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 0.0, 1.0],
    [0.0, -1.0, 0.0],
  ]),
  p=[-0.1, -0.2, 0.104],
)

In [65]:
result = InvKinProg(context, pose)

IK success


In [58]:
result.GetSolution()

array([-1.72004007,  0.55612755, -0.09451926, -2.43337601, -0.23156651,
        1.43209801, -0.86914392,  0.        ,  0.        ])